In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from my_functions import combokey_converter

%matplotlib inline
sns.set_style('whitegrid')
plt.rc('axes', titlesize = 14, titleweight = 'bold', labelweight = 'bold')

In [2]:
raw = pd.read_csv('../filtered_data/00_crdc_1516_initial.csv', 
                        dtype = {'LEAID':np.object})

In [3]:
filtered = pd.read_csv('../filtered_data/04_filter_final.csv', dtype = {'LEAID':np.object})

In [4]:
raw['COMBOKEY'] = combokey_converter.convert(raw, 'LEAID', 'SCHID')

In [5]:
filtered_combo = filtered[['COMBOKEY','LAT1516']]

In [6]:
filtered_raw_joined = pd.merge(raw, filtered_combo, on = 'COMBOKEY', how = 'left')
filtered_from_raw = filtered_raw_joined.copy()
filtered_from_raw = filtered_from_raw[filtered_from_raw.LAT1516.isnull()]
filtered_from_raw = filtered_from_raw.drop('LAT1516', axis = 1)

In [7]:
filtered_from_raw.columns.values

array(['LEA_STATE', 'LEA_STATE_NAME', 'LEAID', 'LEA_NAME', 'SCHID',
       'SCH_NAME', 'COMBOKEY', 'JJ', 'SCH_GRADE_PS', 'SCH_GRADE_KG',
       'SCH_GRADE_G01', 'SCH_GRADE_G02', 'SCH_GRADE_G03', 'SCH_GRADE_G04',
       'SCH_GRADE_G05', 'SCH_GRADE_G06', 'SCH_GRADE_G07', 'SCH_GRADE_G08',
       'SCH_GRADE_G09', 'SCH_GRADE_G10', 'SCH_GRADE_G11', 'SCH_GRADE_G12',
       'SCH_GRADE_UG', 'SCH_UGDETAIL_HS', 'SCH_STATUS_SPED',
       'SCH_STATUS_MAGNET', 'SCH_STATUS_CHARTER', 'SCH_STATUS_ALT',
       'SCH_ENR_HI_M', 'SCH_ENR_HI_F', 'SCH_ENR_AM_M', 'SCH_ENR_AM_F',
       'SCH_ENR_AS_M', 'SCH_ENR_AS_F', 'SCH_ENR_HP_M', 'SCH_ENR_HP_F',
       'SCH_ENR_BL_M', 'SCH_ENR_BL_F', 'SCH_ENR_WH_M', 'SCH_ENR_WH_F',
       'SCH_ENR_TR_M', 'SCH_ENR_TR_F', 'TOT_ENR_M', 'TOT_ENR_F',
       'SCH_ENR_LEP_M', 'SCH_ENR_LEP_F', 'SCH_ENR_IDEA_M',
       'SCH_ENR_IDEA_F', 'SCH_DUAL_IND', 'SCH_DUALENR_HI_M',
       'SCH_DUALENR_HI_F', 'SCH_DUALENR_AM_M', 'SCH_DUALENR_AM_F',
       'SCH_DUALENR_AS_M', 'SCH_DUALENR_AS_F

# Analysis

In [8]:
def missing_value_mapper(value):
    """Converts any negative number into 0, as these negative numbers represent missing/null values"""
    if isinstance(value, int):
        if value < 0:
            return 0
    return value

filtered_from_raw = filtered_from_raw.applymap(missing_value_mapper)

In [9]:
filtered_from_raw['total_enrollment'] = filtered_from_raw['TOT_ENR_M'] + filtered_from_raw['TOT_ENR_F']
from my_functions.extra_functions import hs_enrollment_averager
filtered_from_raw['total_hs_enrollment'] = hs_enrollment_averager(filtered_from_raw)
filtered_from_raw['de_total_enrollment'] = filtered_from_raw['TOT_DUALENR_M'] + filtered_from_raw['TOT_DUALENR_F']

In [10]:
"""How many filtered schools?"""
len(filtered_from_raw)

77693

In [11]:
filtered_from_raw.columns.values

array(['LEA_STATE', 'LEA_STATE_NAME', 'LEAID', 'LEA_NAME', 'SCHID',
       'SCH_NAME', 'COMBOKEY', 'JJ', 'SCH_GRADE_PS', 'SCH_GRADE_KG',
       'SCH_GRADE_G01', 'SCH_GRADE_G02', 'SCH_GRADE_G03', 'SCH_GRADE_G04',
       'SCH_GRADE_G05', 'SCH_GRADE_G06', 'SCH_GRADE_G07', 'SCH_GRADE_G08',
       'SCH_GRADE_G09', 'SCH_GRADE_G10', 'SCH_GRADE_G11', 'SCH_GRADE_G12',
       'SCH_GRADE_UG', 'SCH_UGDETAIL_HS', 'SCH_STATUS_SPED',
       'SCH_STATUS_MAGNET', 'SCH_STATUS_CHARTER', 'SCH_STATUS_ALT',
       'SCH_ENR_HI_M', 'SCH_ENR_HI_F', 'SCH_ENR_AM_M', 'SCH_ENR_AM_F',
       'SCH_ENR_AS_M', 'SCH_ENR_AS_F', 'SCH_ENR_HP_M', 'SCH_ENR_HP_F',
       'SCH_ENR_BL_M', 'SCH_ENR_BL_F', 'SCH_ENR_WH_M', 'SCH_ENR_WH_F',
       'SCH_ENR_TR_M', 'SCH_ENR_TR_F', 'TOT_ENR_M', 'TOT_ENR_F',
       'SCH_ENR_LEP_M', 'SCH_ENR_LEP_F', 'SCH_ENR_IDEA_M',
       'SCH_ENR_IDEA_F', 'SCH_DUAL_IND', 'SCH_DUALENR_HI_M',
       'SCH_DUALENR_HI_F', 'SCH_DUALENR_AM_M', 'SCH_DUALENR_AM_F',
       'SCH_DUALENR_AS_M', 'SCH_DUALENR_AS_F

## Get NCES information and join with match_schools_with_de

In [12]:
nces_1516_full = pd.read_csv('../filtered_data/01_nces_1516_initial_combined_ccd.csv')
nces_important_columns = ['combokey', 'SCH_TYPE', 'LEVEL', 'VIRTUAL', 'GSLO', 'GSHI',
       'NMCNTY15', 'LOCALE15', 'LAT1516', 'LON1516']
nces_1516_full = nces_1516_full[nces_important_columns]

/Users/alijahoconnor/Box Sync/OCR/Data_and_Analysis/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
filtered_from_raw_with_nces = pd.merge(filtered_from_raw, nces_1516_full, left_on = 'COMBOKEY', 
                                       right_on = 'combokey', how = 'left', suffixes = ('', '_nces'))

In [14]:
non_matching_total = filtered_from_raw_with_nces[filtered_from_raw_with_nces.LEVEL.isnull()]
matching_schools_total = filtered_from_raw_with_nces[filtered_from_raw_with_nces.LEVEL.notnull()]

**Recover some of the non-nces-matching schools**

In [15]:
"""Passing in my recovered_schools from 00_initial_filter"""
%store -r recovered_schools_all

In [16]:
recovered_schools_all = recovered_schools_all.reset_index()
nces_columns = ['COMBOKEY', 'SCH_TYPE', 'LEVEL', 'VIRTUAL', 'GSLO', 'GSHI',
       'NMCNTY15', 'LOCALE15', 'LAT1516', 'LON1516']
recovered_schools_all_nces = recovered_schools_all[nces_columns]

In [17]:
non_matching_schools_total = non_matching_total.drop(nces_important_columns,axis = 1)

In [18]:
non_matching_schools_total

,LEA_STATE,LEA_STATE_NAME,LEAID,LEA_NAME,SCHID,SCH_NAME,COMBOKEY,JJ,SCH_GRADE_PS,SCH_GRADE_KG,...,SCH_IBENR_TR_F,TOT_IBENR_M,TOT_IBENR_F,SCH_IBENR_LEP_M,SCH_IBENR_LEP_F,SCH_IBENR_IDEA_M,SCH_IBENR_IDEA_F,total_enrollment,total_hs_enrollment,de_total_enrollment
3,AL,ALABAMA,100002,Alabama Youth Services,99995,AUTAUGA CAMPUS,='010000299995',Yes,No,No,...,0,0,0,0,0,0,0,38,19,0
58,AL,ALABAMA,100021,Brewer-Porch Childrens Center,1832,Brewer Porch Children's Ctr,='010002101832',No,Yes,Yes,...,0,0,0,0,0,0,0,162,27,0
59,AL,ALABAMA,100022,Sequel TSI Courtland,1833,Three Springs Courtland Sch,='010002201833',No,No,No,...,0,0,0,0,0,0,0,45,26,0
60,AL,ALABAMA,100023,Sequel Madison TSI,1834,Sequel Madison TSI,='010002301834',No,No,No,...,0,0,0,0,0,0,0,57,33,0
61,AL,ALABAMA,100024,Sequel TSI- New Beginnings School/ Owens Cross...,1835,Three Springs New Beginnings,='010002401835',No,No,No,...,0,0,0,0,0,0,0,41,27,0
62,AL,ALABAMA,100026,Sequel TSI Tuskegee,2352,Sequel TSI Tuskegee LLC,='010002602352',No,No,No,...,0,0,0,0,0,0,0,42,28,0
67,AL,ALABAMA,100033,Glenwood Mental Health Services,99998,Allan Cott,='010003399998',No,No,No,...,0,0,0,0,0,0,0,35,18,0
68,AL,ALABAMA,100033,Glenwood Mental Health Services,99999,Lakeview,='010003399999',No,No,Yes,...,0,0,0,0,0,0,0,18,0,0
69,AL,ALABAMA,100035,Learning Tree Inc,1845,Learning Tree Inc Semmes,='010003501845',No,Yes,No,...,0,0,0,0,0,0,0,63,0,0
70,AL,ALABAMA,100035,Learning Tree Inc,1846,Learning Tree Inc Webb,='010003501846',No,Yes,No,...,0,0,0,0,0,0,0,42,0,0


In [19]:
recovered_non_matchings = pd.merge(non_matching_schools_total, recovered_schools_all_nces, on='COMBOKEY')

In [20]:
filtered_from_raw = matching_schools_total.append(recovered_non_matchings)

In [21]:
total_nces_matching_schools = len(filtered_from_raw)
total_nces_non_matching_schools = len(filtered_from_raw_with_nces) - len(filtered_from_raw)
print(total_nces_matching_schools, 'Schools Match and',  total_nces_non_matching_schools, 'Do not match with NCES')

74537 Schools Match and 3156 Do not match with NCES


In [22]:
# filtered_from_raw_with_nces.to_csv('../filtered_data/04_filtered_out_schools.csv')

### All Filtered-out Schools

### Filtered-Out Schools with DE Students

In [23]:
match_schools_with_de = filtered_from_raw.copy()
match_schools_with_de = match_schools_with_de[match_schools_with_de.de_total_enrollment > 0]

In [24]:
"""How many filtered schools have DE students?"""
len(match_schools_with_de)

873

In [25]:
first_columns = ['SCH_NAME', 'de_total_enrollment', 'total_enrollment', 'total_hs_enrollment']
reorder = first_columns + [c for c in match_schools_with_de.columns if c not in first_columns]
match_schools_with_de = match_schools_with_de[reorder].sort_values('de_total_enrollment', ascending = False)
match_schools_with_de

,SCH_NAME,de_total_enrollment,total_enrollment,total_hs_enrollment,COMBOKEY,GSHI,GSLO,JJ,LAT1516,LEAID,...,TOT_APENR_F,TOT_APENR_M,TOT_DUALENR_F,TOT_DUALENR_M,TOT_ENR_F,TOT_ENR_M,TOT_IBENR_F,TOT_IBENR_M,VIRTUAL,combokey
5566,John F. Kennedy High,540,641,641,='060985011712',12,10,No,33.9173,609850,...,0,0,321,219,377,264,0,0,No,='060985011712'
26248,Lowell Senior High School,520,1196,957,='181146001852',12,8,No,41.2914,1811460,...,147,99,200,320,573,623,0,0,No,='181146001852'
26383,Ben Davis Ninth Grade Center,496,1275,1275,='181281002122',9,9,No,39.783,1812810,...,0,0,250,246,627,648,0,0,No,='181281002122'
76794,Oak Creek High,443,2086,2086,='551083001411',12,9,No,42.8867,5510830,...,247,192,224,219,1013,1073,0,0,No,='551083001411'
73952,Tesla STEM High School,395,565,565,='530423003432',12,9,No,47.6487,5304230,...,150,251,136,259,210,355,0,0,Missing,='530423003432'
2154,Lake Havasu High School,376,1843,1843,='040428000433',12,9,No,34.4944,404280,...,0,0,239,137,902,941,0,0,No,='040428000433'
71505,MAGGIE L. WALKER GOV. SCH.,328,745,745,='510006102561',N,N,No,37.5579,5100061,...,267,223,168,160,432,313,0,0,No,='510006102561'
7935,Sherman Oaks Center for Enriched Studies,321,2089,928,='062271009151',12,4,No,34.1849,622710,...,203,210,180,141,1046,1043,0,0,No,='062271009151'
74381,Stahl Junior High,298,856,285,='530696001178',9,5,No,47.1022,5306960,...,0,0,146,152,446,410,0,0,Missing,='530696001178'
68159,MERCEDES EARLY COLLEGE ACADEMY,293,330,330,='483025012441',12,9,No,26.1436,4830250,...,34,23,148,145,179,151,0,0,Missing,='483025012441'


**Total DE Students**

In [26]:
"""Total DE Students"""
total_filtered_out_de_students = filtered_from_raw_with_nces.de_total_enrollment.sum()
de_students_in_NCES_matching_schools =  match_schools_with_de.de_total_enrollment.sum()
de_students_in_NCES_non_matching_schools = total_filtered_out_de_students -\
                                           de_students_in_NCES_matching_schools
print(de_students_in_NCES_matching_schools, 'Students in matching schools.')
print(de_students_in_NCES_non_matching_schools, 'Students in non-matching schools.')
print(str(total_filtered_out_de_students), 'Total DE Students Filtered Out')

25931 Students in matching schools.
2453 Students in non-matching schools.
28384 Total DE Students Filtered Out


**Juvenile Justice**

In [27]:
"""Juvenile Justice Schools - How many schools and DE students?"""
jj_de_students = match_schools_with_de[match_schools_with_de['JJ'] == 'Yes'].de_total_enrollment.sum()
print(match_schools_with_de.JJ.value_counts())
print()
print(str(jj_de_students), "DE students.")

No     861
Yes     12
Name: JJ, dtype: int64

363 DE students.


**Alternative**

In [28]:
"""Alternative Schools - How many schools and DE students?"""
alt_de_students_crdc = match_schools_with_de[match_schools_with_de.SCH_STATUS_ALT == 'Yes'].de_total_enrollment.sum()
print(match_schools_with_de.SCH_STATUS_ALT.value_counts())
print()
print(str(alt_de_students_crdc), "DE students.")

No     539
Yes    334
Name: SCH_STATUS_ALT, dtype: int64

7406 DE students.


**Special Education**

In [29]:
"""Special Education - How many schools and DE students?"""
sped_de_students_crdc = match_schools_with_de[match_schools_with_de.SCH_STATUS_SPED == 'Yes'].de_total_enrollment.sum()
print(match_schools_with_de.SCH_STATUS_SPED.value_counts())
print()
print(str(sped_de_students_crdc), "DE students.")

No     802
Yes     71
Name: SCH_STATUS_SPED, dtype: int64

2827 DE students.


**Grade Ranges**

In [30]:
def school_grade_range(df):
    """Calculates the number of schools in a given DataFrame (df) of a particular distribution of grades:
        Example:  If a school (in df) offers grades 9,10,11,12 exclusively, they are placed in the '9-12' 
                    category. """
    df_grade_range = []
    for index, row in df.iterrows():
        grade_range = []

        if row.SCH_GRADE_G12 == 'Yes':
            grade_range.append(12)
        if row.SCH_GRADE_G11 == 'Yes':
            grade_range.append(11)
        if row.SCH_GRADE_G10 == 'Yes':
            grade_range.append(10)
        if row.SCH_GRADE_G09 == 'Yes':
            grade_range.append(9)
        if row.SCH_GRADE_G08 == 'Yes':
            grade_range.append(8)
        if row.SCH_GRADE_G07 == 'Yes':
            grade_range.append(7)
        if row.SCH_GRADE_G06 == 'Yes':
            grade_range.append(6)
        if row.SCH_GRADE_G05 == 'Yes':
            grade_range.append(5)
        if row.SCH_GRADE_G04 == 'Yes':
            grade_range.append(4)
        if row.SCH_GRADE_G03 == 'Yes':
            grade_range.append(3)
        if row.SCH_GRADE_G02 == 'Yes':
            grade_range.append(2)
        if row.SCH_GRADE_G01 == 'Yes':
            grade_range.append(1)
        if row.SCH_GRADE_KG == 'Yes':
            grade_range.append('kg')
        if row.SCH_GRADE_PS == 'Yes':
            grade_range.append('pk')
        
        if grade_range == [12,11,10,9,8,7,6,5,4,3,2,1,'kg','pk']:
            df_grade_range.append('pk-12')
        elif grade_range == [12,11,10,9,8,7,6,5,4,3,2,1,'kg']:
            df_grade_range.append('kg-12')
        elif grade_range == [12,11,10,9,8,7,6,5]:
            df_grade_range.append('05-12')
        elif grade_range == [12,11,10,9,8,7,6]:
            df_grade_range.append('06-12')
        elif grade_range == [12,11,10,9,8,7]:
            df_grade_range.append('07-12')
        elif grade_range == [12,11,10,9,8]:
            df_grade_range.append('08-12')
        elif grade_range == [12,11,10,9]:
            df_grade_range.append('09-12')
        elif grade_range == [11,10,9]:
            df_grade_range.append('09-11')
        elif grade_range == [10,9]:
            df_grade_range.append('09-10')
        elif grade_range == [12,11,10]:
            df_grade_range.append('10-12')
        elif grade_range == [12,11]:
            df_grade_range.append('11-12')
        elif grade_range == [9]:
            df_grade_range.append('9-only')
        elif grade_range == [10]:
            df_grade_range.append('10-only')
        elif grade_range == [11]:
            df_grade_range.append('11-only')
        elif grade_range == [12]:
            df_grade_range.append('12-only')
        else:
            df_grade_range.append('other')
    return pd.DataFrame(df_grade_range, columns=['grade_range'])

In [31]:
de_stud_school_grade_range_list = school_grade_range(match_schools_with_de).join(match_schools_with_de['de_total_enrollment'].reset_index(drop = True), how = 'outer')

de_stud_school_grade_range_dist = de_stud_school_grade_range_list['grade_range'].value_counts().reset_index().sort_values('index').set_index('index')
de_stud_school_grade_range_dist['pct_of_schools'] = round(de_stud_school_grade_range_dist['grade_range'] / len(de_stud_school_grade_range_list), 3)

de_stud_school_grade_range_enrollments = pd.DataFrame(de_stud_school_grade_range_list.groupby('grade_range')['de_total_enrollment'].sum())

de_stud_school_grade_range_dist.join(de_stud_school_grade_range_enrollments).rename({'grade_range': '# schools'}, axis = 1)

,# schools,pct_of_schools,de_total_enrollment
index,,,
05-12,6,0.007,260
06-12,72,0.082,884
07-12,55,0.063,1481
08-12,25,0.029,1062
09-10,15,0.017,1340
09-11,1,0.001,44
09-12,355,0.407,11734
10-12,52,0.060,1249
11-12,20,0.023,835


**Filtered Schools that match with NCES**

**LEVEL**<br>
1 = Primary (low grade = PK through 03; high grade = PK through 08), 2 = Middle (low grade = 04 through 07; high grade = 04 through 09), 3 = High (low grade = 07 through 12; high grade = 12 only), 4 = Other (any other configuration not falling within the above three categories;including ungraded), N = Not applicable


In [32]:
"""How many schools and de students in each LEVEL of school"""
print(match_schools_with_de.LEVEL.value_counts().reset_index().sort_values('index').set_index('index'))
print()
print(str(match_schools_with_de.groupby('LEVEL').de_total_enrollment.sum()), 'DE students.')

       LEVEL
index       
1         22
2         22
3        466
4        338
N         25

LEVEL
1      291
2     1601
3    15214
4     7796
N     1029
Name: de_total_enrollment, dtype: int64 DE students.


In [33]:
level_1_2_N_DE_Students = match_schools_with_de[(match_schools_with_de.LEVEL == '1') | 
                                               (match_schools_with_de.LEVEL == '2') | 
                                               (match_schools_with_de.LEVEL == 'N')].de_total_enrollment.sum()
print(level_1_2_N_DE_Students, 'DE students in Schools reported as LEVELs 1, 2, or N')

2921 DE students in Schools reported as LEVELs 1, 2, or N


**Virtual Schools**

In [34]:
virtual_de_students = match_schools_with_de[match_schools_with_de['VIRTUAL'] == 'Yes'].de_total_enrollment.sum()
print(match_schools_with_de.VIRTUAL.value_counts())
print()
print(match_schools_with_de.groupby('VIRTUAL').de_total_enrollment.sum())

No         544
Missing    181
Yes        148
Name: VIRTUAL, dtype: int64

VIRTUAL
Missing     7744
No         15386
Yes         2801
Name: de_total_enrollment, dtype: int64


**School Type**<br>
1 = Regular school, 2 = Special education school, 3 = Vocational school, 4 = Other/alternative school


In [35]:
sped_de_students_nces = match_schools_with_de[match_schools_with_de['SCH_TYPE'] == 2].de_total_enrollment.sum()
alt_de_students_nces = match_schools_with_de[match_schools_with_de['SCH_TYPE'] == 4].de_total_enrollment.sum()
print(match_schools_with_de.SCH_TYPE.value_counts())
print()
print(match_schools_with_de.groupby('SCH_TYPE').de_total_enrollment.sum())

4.0    464
1.0    350
2.0     49
3.0     10
Name: SCH_TYPE, dtype: int64

SCH_TYPE
1.0    13040
2.0     1167
3.0      498
4.0    11226
Name: de_total_enrollment, dtype: int64


**Students in Schools without offering 11th or 12th grade**

In [36]:
from my_functions.extra_functions import students_in_11_or_12
match_schools_with_de['Students_in_11_12'] = match_schools_with_de.apply(lambda row: students_in_11_or_12(row['SCH_GRADE_G11'], row['SCH_GRADE_G12']), axis = 1)

In [37]:
de_students_schools_without_11_12 = match_schools_with_de[match_schools_with_de.Students_in_11_12 == 'No']\
                                        .de_total_enrollment.sum()
match_schools_with_de.Students_in_11_12.value_counts()
print(str(de_students_schools_without_11_12), 'de students in schools that do not offer grades 11 or 12.')

4553 de students in schools that do not offer grades 11 or 12.


**Filtered schools with the most DE-students**

In [38]:
with pd.option_context('display.max_rows', 125):
    display(match_schools_with_de.sort_values('de_total_enrollment', ascending=False).head(125))

,SCH_NAME,de_total_enrollment,total_enrollment,total_hs_enrollment,COMBOKEY,GSHI,GSLO,JJ,LAT1516,LEAID,...,TOT_APENR_M,TOT_DUALENR_F,TOT_DUALENR_M,TOT_ENR_F,TOT_ENR_M,TOT_IBENR_F,TOT_IBENR_M,VIRTUAL,combokey,Students_in_11_12
5566,John F. Kennedy High,540,641,641,='060985011712',12,10,No,33.9173,609850,...,0,321,219,377,264,0,0,No,='060985011712',Yes
26248,Lowell Senior High School,520,1196,957,='181146001852',12,8,No,41.2914,1811460,...,99,200,320,573,623,0,0,No,='181146001852',Yes
26383,Ben Davis Ninth Grade Center,496,1275,1275,='181281002122',9,9,No,39.783,1812810,...,0,250,246,627,648,0,0,No,='181281002122',No
76794,Oak Creek High,443,2086,2086,='551083001411',12,9,No,42.8867,5510830,...,192,224,219,1013,1073,0,0,No,='551083001411',Yes
73952,Tesla STEM High School,395,565,565,='530423003432',12,9,No,47.6487,5304230,...,251,136,259,210,355,0,0,Missing,='530423003432',Yes
2154,Lake Havasu High School,376,1843,1843,='040428000433',12,9,No,34.4944,404280,...,0,239,137,902,941,0,0,No,='040428000433',Yes
71505,MAGGIE L. WALKER GOV. SCH.,328,745,745,='510006102561',N,N,No,37.5579,5100061,...,223,168,160,432,313,0,0,No,='510006102561',Yes
7935,Sherman Oaks Center for Enriched Studies,321,2089,928,='062271009151',12,4,No,34.1849,622710,...,210,180,141,1046,1043,0,0,No,='062271009151',Yes
74381,Stahl Junior High,298,856,285,='530696001178',9,5,No,47.1022,5306960,...,0,146,152,446,410,0,0,Missing,='530696001178',No
68159,MERCEDES EARLY COLLEGE ACADEMY,293,330,330,='483025012441',12,9,No,26.1436,4830250,...,23,148,145,179,151,0,0,Missing,='483025012441',Yes


In [39]:
# match_schools_with_de.to_csv('../filtered_data/04_filtered_out_match_schools_with_de.csv')

# Where are the Filtered Students?

In [42]:
de_student_list = [total_filtered_out_de_students, de_students_in_NCES_matching_schools, de_students_in_NCES_non_matching_schools,
                              jj_de_students, alt_de_students_crdc, sped_de_students_crdc, de_students_schools_without_11_12,
                              virtual_de_students, sped_de_students_nces, alt_de_students_nces, level_1_2_N_DE_Students]
filtered_list = ['Total Filtered Out', 'NCES-Matching', 'NCES-Non-Matching', 'Juvenile Justice (CRDC)', 'Alternative Education (CRDC)',
                     'Special Education (CRDC)', 'Schools w/o 11th/12th grade (CRDC)', 'Virtual (NCES)', 
                     'Special Education (NCES)', 'Alternative/Other (NCES)', 'Schools Reporting as Elementary/Middle/Other (NCES)']

pd.DataFrame({'DE Students': de_student_list},
            index = filtered_list)

,DE Students
Total Filtered Out,28384
NCES-Matching,25931
NCES-Non-Matching,2453
Juvenile Justice (CRDC),363
Alternative Education (CRDC),7406
Special Education (CRDC),2827
Schools w/o 11th/12th grade (CRDC),4553
Virtual (NCES),2801
Special Education (NCES),1167
Alternative/Other (NCES),11226
